In [ ]:
# Install required packages
!pip install -q ultralytics flask flask-cors
!npm install -g localtunnel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.9 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧
added 22 packages in 3s
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧

In [ ]:
"""
Parking Detector Testing Script
Test your trained YOLO model on images in a separate Colab environment
"""

# Step 1: Install ultralytics
print(" Installing ultralytics...")
import subprocess
subprocess.run(["pip", "install", "-q", "ultralytics"], check=True)
print(" Installation complete!\n")

from ultralytics import YOLO
import cv2
from google.colab.patches import cv2_imshow
from google.colab import files
import numpy as np

# =============================================================================
# CONFIGURATION
# =============================================================================

# Ask the user to upload their trained YOLO model file (e.g., best.pt or last.pt).
# This is useful when the script is run in a fresh Colab that does not have the model on disk.
print("=" * 70)
print(" UPLOAD YOUR TRAINED MODEL")
print("=" * 70)
print("Please upload your 'best.pt' or 'last.pt' file from training")
print("(Usually located in: /content/parking_dataset/runs/parking_detector/weights/)")

# files.upload() opens a file picker in Colab and returns a dict of {filename: file_bytes}.
uploaded = files.upload()

# Take the first (and usually only) uploaded file name as our model path.
model_path = list(uploaded.keys())[0]
print(f" Model uploaded: {model_path}\n")

# =============================================================================
# LOAD MODEL
# =============================================================================

print("=" * 70)
print("Loading Model")
print("=" * 70)

# Create a YOLO object from the uploaded .pt file so it can be used for inference.
model = YOLO(model_path)

# Confirm successful load and print out available class names.
print(f" Model loaded successfully!")
print(f"   Classes: {model.names}")

 Installing ultralytics...
 Installation complete!

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
 UPLOAD YOUR TRAINED MODEL
Please upload your 'best.pt' or 'last.pt' file from training
(Usually located in: /content/parking_dataset/runs/parking_detector/weights/)


Saving best.pt to best.pt
 Model uploaded: best.pt

Loading Model
 Model loaded successfully!
   Classes: {0: 'Free', 1: 'Occupied'}


In [ ]:
# Move model to standard location for API to use
import shutil
shutil.move(model_path, '/content/best.pt')

'/content/best.pt'

In [ ]:
from flask import Flask, request, jsonify, send_file
from flask_cors import CORS
from ultralytics import YOLO
import numpy as np
import cv2, os, threading, time

# Initialize Flask app and enable CORS so a separate frontend (e.g., React Native / web)
# can call this API from another origin.
app = Flask(__name__)
CORS(app)

# Load YOLO model once at startup so it can be reused for all incoming requests.
print("Reloading model for API...")
api_model = YOLO('/content/best.pt')
print("API model loaded!")

# Path on the Colab filesystem where the last annotated image will be saved.
LAST_PATH = "/tmp/last.jpg"

# draw_detections:
#   - Draws bounding boxes on the input image for each detection.
#   - Uses thicker green boxes and labels for "free" / "open" type classes.
#   - Uses thin red-ish boxes (no label) for all other classes.
def draw_detections(img, result):
    names = result.names
    boxes = result.boxes

    # Iterate over detected bounding boxes, their classes, and confidences.
    for xyxy, cls, conf in zip(boxes.xyxy.cpu().numpy(),
                               boxes.cls.cpu().numpy(),
                               boxes.conf.cpu().numpy()):
        # Convert coordinates to integers for OpenCV drawing functions.
        x1, y1, x2, y2 = map(int, xyxy)
        lbl = names[int(cls)].lower()

        # Highlight free/open/vacant spots with a green box and label.
        if lbl in {"free","open","vacant"}:
            color=(0,200,0); thickness=3 #Thick Green outline for open
            txt=f"Free {conf:.2f}"

            # Measure text size to draw a filled rectangle behind the label.
            (tw, th), _ = cv2.getTextSize(txt, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
            y0 = max(0, y1 - th - 6)

            # Draw a semi-transparent background for the label.
            overlay = img.copy()
            cv2.rectangle(overlay,(x1,y0),(x1+tw+8,y1),color,-1)
            cv2.addWeighted(overlay,0.35,img,0.65,0,img)

            # Put the label text above the bounding box.
            cv2.putText(img,txt,(x1+4,y1-4),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,255,255),2)
        else:
            # Non-free spots (e.g., occupied) get a thinner red box.
            color=(50,50,200); thickness=1

        cv2.rectangle(img,(x1,y1),(x2,y2),color,thickness)

    #Return the modified image
    return img

# Sets of class-label names that will be interpreted as "free" vs "occupied".
FREE_LABELS = {"free","open","vacant","empty","available"}
OCC_LABELS  = {"occupied","taken","parked","blocked"}

# count_spots:
#   - Counts how many detections belong to free vs occupied label sets.
#   - Returns (open_cnt, occ_cnt).
def count_spots(result):
    names = result.names
    open_cnt = 0
    occ_cnt  = 0

    # Iterate over each class index to map it with the proper label.
    for cls in result.boxes.cls.cpu().numpy().astype(int):
        lbl = names[int(cls)].lower()
        if lbl in FREE_LABELS:
            open_cnt += 1
        elif lbl in OCC_LABELS:
            occ_cnt  += 1
    return open_cnt, occ_cnt

# /detect:
#   - POST endpoint that accepts an image file in form field "image".
#   - Runs YOLO on the image, counts free/occupied spots, saves an annotated copy.
#   - Returns JSON with counts and a success flag.
@app.post("/detect")
def detect_route():
    # Get uploaded file from form-data.
    f = request.files.get("image")
    if f is None:
        return jsonify(error='missing file field "image"'), 400

    # Read raw bytes from the uploaded file.
    data = f.read()
    if not data:
        return jsonify(error="empty upload"), 400

    # Decode bytes into an OpenCV BGR image.
    arr = np.frombuffer(data, np.uint8)
    img = cv2.imdecode(arr, cv2.IMREAD_COLOR)
    if img is None:
        return jsonify(error="decode failed (bad image bytes)"), 400

    # Count free and occupied spots from detection results.
    result = api_model(img, conf=0.35, iou=0.50)[0]
    open_cnt, occ_cnt = count_spots(result)

    # Draw bounding boxes and labels on a copy of the image.
    annotated = draw_detections(img.copy(), result)

    # Save annotated image so it can be downloaded via /get_image.
    cv2.imwrite(LAST_PATH, annotated)

    # Calculate the total spots.
    total = open_cnt + occ_cnt

    # Expose multiple key names to match multiple possibkle frontends.
    return jsonify(
        ok=True,
        free=open_cnt,
        occupied=occ_cnt,
        total=total,
        openCount=open_cnt,
        occupiedCount=occ_cnt
    )

# /get_image:
#   - GET endpoint that returns the last annotated image (JPEG).
#   - Returns 404 with JSON error if no image has been processed yet.
@app.get("/get_image")
def get_image():
    if not os.path.exists(LAST_PATH):
        return jsonify({'error': 'No image available'}), 404
    return send_file(LAST_PATH, mimetype='image/jpeg')

# /status:
#   - Simple health-check endpoint for the API.
#   - Returns model name and class dictionary from YOLO.
@app.get("/status")
def status():
    return jsonify({'status':'running','model':'best.pt','classes':api_model.names})

# run_flask:
#   - Helper function to run the Flask app on all interfaces on port 5000.
#   - Used as the target for a background thread so the notebook cell can continue.
def run_flask():
    app.run(host='0.0.0.0', port=5000, threaded=True)

# Start Flask server in a background daemon thread so that Colab cell does not block.
flask_thread = threading.Thread(target=run_flask, daemon=True)
flask_thread.start()

# Give Flask some time to start up before launching localtunnel.
time.sleep(3)

# Start localtunnel to expose port 5000 to the internet.
# The generated public URL is written into /content/url.txt.
get_ipython().system_raw('lt --port 5000 > /content/url.txt 2>&1 &')

# Allow local tunnel to initialize and write the URL file.
time.sleep(8)


🔄 Reloading model for API...
✅ API model loaded!
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
# Get the flask URL
!cat /content/url.txt


your url is: https://rare-deer-stand.loca.lt
